In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
#Use this file path for pc
filepath = r"D:\gradschool\Capstone"
#Use this file path for mac 
# filepath = r"/Users/jacksonjoyner/Desktop/MSBA/Capstone/UseThisData/"
os.chdir(filepath)

In [3]:
playerdf = pd.read_csv("PCATest1.csv")
playerdf = playerdf.rename(columns ={"total cs" : "total_cs"})
playerdf["total_cs"]

0         307
1         138
2         361
3         333
4          33
         ... 
391557    274
391558    167
391559    318
391560    314
391561     39
Name: total_cs, Length: 391562, dtype: int64

Part of this data was incorrect, so I corrected it in excel. For KDA, the calculation is (Kills + Assists)/Deaths. If Deaths is 0, you get an infinite number, so I changed it in those instances to be (kills + Assists) / 1. This is common practice in industry. 

From testing, this is the most accurate model I can create

In [21]:
playerdf

,playername,position,teamname,kills,deaths,assists,doublekills,firstbloodkill,firstbloodassist,firstbloodvictim,...,D%,earnedgoldshare,damageshare,cspm,damagetakenperminute,earnedgold,earnedgpm,PCA15,PCAShare,PCApm
0,Cruzer,top,Dignitas,2,2,7,0.0,0.0,0.0,1.0,...,0.181818,0.206422,0.183627,7.9602,837.7960,10465.0,271.3483,0.823107,-0.047352,0.818600
1,Crumbz,jng,Dignitas,0,3,10,0.0,0.0,0.0,0.0,...,0.272727,0.161569,0.082062,3.5782,666.8453,8191.0,212.3855,-0.730705,-1.299864,-0.707233
2,scarra,mid,Dignitas,3,2,8,0.0,0.0,0.0,0.0,...,0.181818,0.230584,0.392236,9.3604,277.1046,11690.0,303.1115,1.303923,1.756214,1.420342
3,Imaqtpie,bot,Dignitas,9,1,6,2.0,0.0,0.0,0.0,...,0.090909,0.264175,0.238274,8.6344,393.0337,13393.0,347.2688,0.126891,1.006983,1.644049
4,KiWiKiD,sup,Dignitas,3,3,9,0.0,0.0,0.0,0.0,...,0.272727,0.137249,0.103801,0.8557,213.7338,6958.0,180.4149,-2.997363,-1.415598,-1.614021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391557,Rock,top,Hong Kong Attitude,6,4,3,1.0,0.0,1.0,0.0,...,0.153846,0.273646,0.329602,8.0273,1232.1680,8993.0,263.4668,0.832963,1.784049,0.764343
391558,Patience,jng,Hong Kong Attitude,0,7,2,0.0,0.0,1.0,0.0,...,0.269231,0.138549,0.128556,4.8926,908.9355,4553.0,133.3887,-0.628094,-1.219308,-1.103865
391559,B1ven,mid,Hong Kong Attitude,1,4,4,0.0,0.0,0.0,0.0,...,0.153846,0.253016,0.274324,9.3164,501.3867,8315.0,243.6035,1.912411,1.145145,0.884427
391560,MnM,bot,Hong Kong Attitude,2,4,4,0.0,0.0,0.0,0.0,...,0.153846,0.252834,0.173123,9.1992,505.2832,8309.0,243.4277,0.643232,0.400768,0.855998


In [4]:
variables = ['kills','deaths','assists','PCA15','PCAShare','PCApm']
x = playerdf[variables].to_numpy() 
y = playerdf["result"]

In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [6]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression(max_iter = 100000)

In [7]:
# fit the model with data
logreg = logreg.fit(x_train,y_train)

#
y_pred=logreg.predict(x_test)
logreg.coef_, logreg.intercept_

(array([[ 0.46041754, -0.8406109 ,  0.42373877, -0.54574548, -2.37918379,
          2.89146595]]),
 array([-1.60072879]))

In [8]:
logreg.score(x_test,y_test)

0.9013596755575078

In [9]:
y_pred

array([1, 1, 1, ..., 0, 1, 0], dtype=int64)

In [10]:
#https://www.datasklr.com/ols-least-squares-regression/multicollinearity
model = sm.OLS(y_train, sm.add_constant(x_train)).fit()
Y_pred = model.predict(sm.add_constant(x_test))
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                 result   R-squared:                       0.618
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                 7.907e+04
Date:                Mon, 28 Mar 2022   Prob (F-statistic):               0.00
Time:                        11:55:12   Log-Likelihood:                -71962.
No. Observations:              293671   AIC:                         1.439e+05
Df Residuals:                  293664   BIC:                         1.440e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3357      0.001    229.306      0.0